In [10]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

from utils import seed_everything, set_device, get_train_val_dataloaders, train
from model import FeedForwardRegressionNet

from torch.utils.data import TensorDataset
from torch.optim import AdamW

In [11]:
n_jobs = 1000
results = []

for i in range(n_jobs):
    results.append(pickle.load(open(f"results/training_stats_{i}", "rb")))

In [12]:
df = pd.DataFrame(results).sort_values("Valid. Loss")

In [13]:
df[0:10]

,Training Loss,Valid. Loss,Valid. CI,Training Time,Validation Time,hidden_dims,learning_rate,embed_dim0,embed_dim1,epochs,job_id,weight_decay
462,6064.796328,5033.432192,724.901440,0:00:03,0:00:00,"[64, 128]",0.010,16,4,32,462,0.0030
680,6211.431389,5263.054191,721.012220,0:00:02,0:00:00,"[64, 128]",0.010,4,32,32,680,0.0003
753,5520.396385,5797.777871,993.686591,0:00:02,0:00:00,"[64, 128]",0.010,16,16,32,753,0.0010
600,6151.538141,5896.051965,1197.892402,0:00:02,0:00:00,"[128, 8]",0.010,4,16,32,600,0.0030
438,6514.384305,5931.405604,675.170082,0:00:02,0:00:00,"[128, 16]",0.010,4,4,32,438,0.0000
959,5630.662366,6357.435193,946.263469,0:00:01,0:00:00,"[128, 16]",0.010,16,16,32,959,0.0010
41,6477.138495,6382.839544,848.396084,0:00:03,0:00:00,"[128, 64]",0.010,16,32,32,41,0.0000
683,6739.437005,6529.928876,933.661296,0:00:02,0:00:00,"[128, 64]",0.010,16,32,32,683,0.0003
610,7261.850720,6559.227511,1000.420066,0:00:01,0:00:00,"[64, 128]",0.003,4,32,32,610,0.0003
209,6881.805862,6619.525103,926.978175,0:00:03,0:00:00,"[128, 16]",0.010,8,16,32,209,0.0003


In [41]:
best_hyperparameters = {
    "hidden_dims": [128, 64, 8],
    "learning_rate": 0.003,
    "embed_dim0": 16,
    "embed_dim1": 8,
    "epochs": 32,
    "weight_decay" : 0.003
}

pickle.dump(best_hyperparameters, open("best_hyperparameters.pkl", "wb"))

In [42]:
hyperparameters = pickle.load(open("best_hyperparameters.pkl", "rb"))

# hyperparameters = {
#     "hidden_dims": [64, 128],
#     "learning_rate": 0.01,
#     "embed_dim0": 16,
#     "embed_dim1": 4,
#     "epochs": 32,
#     "weight_decay" : 0.003
# }

HIDDEN_DIMS = hyperparameters["hidden_dims"]
LEARNING_RATE = hyperparameters["learning_rate"]
EMBED_DIM0 = hyperparameters["embed_dim0"]
EMBED_DIM1 = hyperparameters["embed_dim1"]
EPOCHS = hyperparameters["epochs"]
WEIGHT_DECAY = hyperparameters["weight_decay"]

batch_size = 64
val_size = 0.1

In [43]:
device = set_device()

X = pickle.load(open("preprocessed_data/X_train_tensor.pkl", "rb"))
y = pickle.load(open("preprocessed_data/y_train_tensor.pkl", "rb"))

dataset = TensorDataset(X, y)

train_dataloader, validation_dataloader = get_train_val_dataloaders(
    dataset, val_size, batch_size
)

No GPU available, using the CPU instead.
45,000 training samples
4,999 validation samples


In [44]:
input_dim = X.shape[1] - 2
vocab_size0 = len(np.unique(X[:, 0]))
vocab_size1 = len(np.unique(X[:, 1]))
output_dim = 1

model = FeedForwardRegressionNet(
    input_dim,
    vocab_size0,
    vocab_size1,
    EMBED_DIM0,
    EMBED_DIM1,
    HIDDEN_DIMS,
    output_dim,
)

In [45]:
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

criterion = nn.MSELoss()

seed_everything(42)

training_stats = train(
    model,
    EPOCHS,
    train_dataloader,
    validation_dataloader,
    criterion,
    optimizer,
    device=device,
    SSY=y.var() * len(y),
)


======== Epoch 1 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:01.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:02.

  Average training loss: 512767.08
  Training epoch took: 0:00:02

Running Validation...
  Validation Loss: 49011.08
  Validation CI: +/-4326.55
  Validatio

  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:02.

  Average training loss: 15048.37
  Training epoch took: 0:00:02

Running Validation...
  Validation Loss: 13981.67
  Validation CI: +/-1796.93
  Validation SSE: 70593216.00
  Validation took: 0:00:00
  Validation R^2: 0.9970073103904724

======== Epoch 9 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:01.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   48

  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:01.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:02.

  Average training loss: 9717.86
  Training epoch took: 0:00:02

Running Validation...
  Validation Loss: 8872.29
  Validation CI: +/-1218.67
  Validation SSE: 44788352.00
  Validation took: 0:00:00
  Validation R^2: 0.9981012940406799

======== Epoch 17 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80

  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:01.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:02.

  Average training loss: 6943.58
  Training epoch took: 0:00:02

Running Validation...
  Validation Loss: 6508.03
  Validation CI: +/-984.94
  Validation SSE: 32817526.00
  Validation took: 0:00:00
  Validation R^2: 0.9986087679862976

======== Ep

  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 5259.60
  Training epoch took: 0:00:02

Running Validation...
  Validation Loss: 5923.96
  Validation CI: +/-791.75
  Validation SSE: 29613872.00
  Validation took: 0:00:00
  Validation R^2: 0.998744547367096

======== Epoch 32 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:01.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  

In [46]:
torch.save(model.state_dict(), "best_model.pt")